# Introdução a Ciência dos Dados - Atividade 2

<span style="position: absolute; top: 10px; right: 10px; background: green; padding: 0.5em; color: white; border-radius: 0.25em; font-weight: bold">Vaux Gomes</span>

## Métodos de Análise Estatística dos Dados

#### Bibliotecas

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Loading

In [6]:
df = pd.read_csv('../data/iphone-olx-ce.csv', index_col=0)
df.shape

(1198, 17)

In [10]:
columns = ['region', 'areaCode', 'eletronicsModel', 'storage', 'color', 'batteryHealth', 'eletronicsCondition']
df[columns] = df[columns].astype('category')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1198 entries, 0 to 1531
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   adDate               1198 non-null   float64 
 1   region               1198 non-null   category
 2   zipcode              1198 non-null   int64   
 3   areaCode             1198 non-null   category
 4   category             1198 non-null   object  
 5   paidPromotion        1198 non-null   bool    
 6   subject              1198 non-null   object  
 7   eletronicsModel      1198 non-null   category
 8   storage              1198 non-null   category
 9   color                1198 non-null   category
 10  batteryHealth        1198 non-null   category
 11  eletronicsCondition  1198 non-null   category
 12  price                1198 non-null   float64 
 13  pictures             1198 non-null   float64 
 14  sellerName           1198 non-null   object  
 15  storageGB            1198 

#### Adjustando Data

In [20]:
df.adDate = pd.to_datetime(df.adDate, unit='s')

### Medidas de Tendência Central

### Medidas de Variabilidade

### Tabela de Frequência e Histograma

### Boxplot

### QQ-Plot

### Correlações

### Teste de Normalidade

### Best Fit Distribution 